In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dataframe_image as dfi

In [5]:
df = pd.read_csv("dataframe/all_summary_statistics.csv")

In [6]:
df.head()

,model,quantile,param,summary_statistic,Cramer-von Mises Distance,Energy Distance,Kullback-Leibler Divergence,Maximum Mean Discrepancy,Wasserstein Distance
0,n0.5_no_smoothing,5%,a,Mean,7.014999,1.240841,1.510549,2.607069,-0.094318
1,n0.5_no_smoothing,5%,a,Median,7.501374,1.078339,1.434653,2.672813,-0.179502
2,n0.5_no_smoothing,5%,a,StDev,2.156312,0.798316,1.414268,1.268097,0.599466
3,n0.5_no_smoothing,5%,a,Lower Bound,2.453933,0.173918,-0.514099,0.642807,-1.115715
4,n0.5_no_smoothing,5%,a,Upper Bound,9.770629,2.836456,4.450571,4.667408,1.118796


In [7]:
median = df[(df["summary_statistic"] == "Median") & (df["quantile"] == "0.1%")].reset_index(drop=True)

In [8]:
median

,model,quantile,param,summary_statistic,Cramer-von Mises Distance,Energy Distance,Kullback-Leibler Divergence,Maximum Mean Discrepancy,Wasserstein Distance
0,n0.5_no_smoothing,0.1%,a,Median,2.245654,0.958425,1.077603,0.958425,0.876752
1,n0.5_no_smoothing,0.1%,b,Median,-1.446457,0.769563,2.823467,0.769563,1.167789
2,n0.5_smoothing,0.1%,a,Median,5.410211,1.006735,1.452414,1.006735,0.830041
3,n0.5_smoothing,0.1%,b,Median,-4.322317,1.039599,2.916627,1.039599,1.750355
4,n0_no_smoothing,0.1%,a,Median,2.056443,1.057923,0.724711,0.914861,0.862194
5,n0_no_smoothing,0.1%,b,Median,-0.992702,1.082663,2.784505,0.949178,1.543359


In [9]:
keep_cols = ["model", "param", "Cramer-von Mises Distance", "Energy Distance", "Kullback-Leibler Divergence",
            "Maximum Mean Discrepancy", "Wasserstein Distance"]

median_table = median[keep_cols]

# Option 1 - A Column for True Value

In [10]:
option1 = median_table.copy()
option1["True Value"] = np.ones(len(median_table))
option1

,model,param,Cramer-von Mises Distance,Energy Distance,Kullback-Leibler Divergence,Maximum Mean Discrepancy,Wasserstein Distance,True Value
0,n0.5_no_smoothing,a,2.245654,0.958425,1.077603,0.958425,0.876752,1.0
1,n0.5_no_smoothing,b,-1.446457,0.769563,2.823467,0.769563,1.167789,1.0
2,n0.5_smoothing,a,5.410211,1.006735,1.452414,1.006735,0.830041,1.0
3,n0.5_smoothing,b,-4.322317,1.039599,2.916627,1.039599,1.750355,1.0
4,n0_no_smoothing,a,2.056443,1.057923,0.724711,0.914861,0.862194,1.0
5,n0_no_smoothing,b,-0.992702,1.082663,2.784505,0.949178,1.543359,1.0


In [21]:
piv_table = pd.pivot_table(option1, values=["True Value", "Cramer-von Mises Distance", "Energy Distance", "Kullback-Leibler Divergence",
                                          "Maximum Mean Discrepancy", "Wasserstein Distance"], index = ["model", "param"])

piv_table = piv_table[["True Value", "Cramer-von Mises Distance", "Energy Distance", "Kullback-Leibler Divergence",
                       "Maximum Mean Discrepancy", "Wasserstein Distance"]]

piv_table

True Value  Cramer-von Mises Distance  \
model             param                                          
n0.5_no_smoothing a             1.0                   2.245654   
                  b             1.0                  -1.446457   
n0.5_smoothing    a             1.0                   5.410211   
                  b             1.0                  -4.322317   
n0_no_smoothing   a             1.0                   2.056443   
                  b             1.0                  -0.992702   

                         Energy Distance  Kullback-Leibler Divergence  \
model             param                                                 
n0.5_no_smoothing a             0.958425                     1.077603   
                  b             0.769563                     2.823467   
n0.5_smoothing    a             1.006735                     1.452414   
                  b             1.039599                     2.916627   
n0_no_smoothing   a             1.057923                     0.724711   
                  b             1.082663                     2.784505   

                         Maximum Mean Discrepancy  Wasserstein Distance  
model             param                                                  
n0.5_no_smoothing a                      0.958425              0.876752  
                  b                      0.769563              1.167789  
n0.5_smoothing    a                      1.006735              0.830041  
                  b                      1.039599              1.750355  
n0_no_smoothing   a                      0.914861              0.862194  
                  b                      0.949178              1.543359

In [29]:
df_styled = piv_table.style.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlOrRd") #adding a gradient based on values in cell
df_styled.format(precision=2)

In [30]:
dfi.export(df_styled,"test.png")

# Option 2 - Columns are Difference between True and Simulated Value

In [14]:
option2 = median_table.copy()
s = median_table.select_dtypes("number").columns
option2[s] = np.round(np.abs(median_table[s]-1), 3)

In [15]:
piv_table_2 = pd.pivot_table(option2, values=["Cramer-von Mises Distance", "Energy Distance", "Kullback-Leibler Divergence",
                                          "Maximum Mean Discrepancy", "Wasserstein Distance"], index = ["model", "param"])

piv_table_2 = piv_table_2[["Cramer-von Mises Distance", "Energy Distance", "Kullback-Leibler Divergence",
                       "Maximum Mean Discrepancy", "Wasserstein Distance"]]

piv_table_2

Cramer-von Mises Distance  Energy Distance  \
model             param                                               
n0.5_no_smoothing a                          1.246            0.042   
                  b                          2.446            0.230   
n0.5_smoothing    a                          4.410            0.007   
                  b                          5.322            0.040   
n0_no_smoothing   a                          1.056            0.058   
                  b                          1.993            0.083   

                         Kullback-Leibler Divergence  \
model             param                                
n0.5_no_smoothing a                            0.078   
                  b                            1.823   
n0.5_smoothing    a                            0.452   
                  b                            1.917   
n0_no_smoothing   a                            0.275   
                  b                            1.785   

                         Maximum Mean Discrepancy  Wasserstein Distance  
model             param                                                  
n0.5_no_smoothing a                         0.042                 0.123  
                  b                         0.230                 0.168  
n0.5_smoothing    a                         0.007                 0.170  
                  b                         0.040                 0.750  
n0_no_smoothing   a                         0.085                 0.138  
                  b                         0.051                 0.543

In [31]:
df_styled_2 = piv_table_2.style.background_gradient(axis=None, vmin=0, vmax=10, cmap="YlGnBu") #adding a gradient based on values in cell
df_styled_2.format(precision=2)

In [33]:
dfi.export(df_styled_2,"test_2.png")